## 测试基于 clickhouse 的策略管理系统接口

---



In [1]:
import sys
sys.path.insert(0, "A:/ZB/git_repo/waditu/czsc")
import dotenv
dotenv.load_dotenv(dotenv_path="A:/ZB/git_repo/waditu/czsc/.env", override=True)

import os
import czsc
import pandas as pd
from zoneinfo import ZoneInfo


czsc.welcome()

database = "czsc_strategy_test"
tz = ZoneInfo("Asia/Shanghai")

## 初始化策略管理数据库

czsc.cwc.initialize(database=database)

C:\Users\zengb\.czsc 目录缓存超过1GB，请适当清理。调用 czsc.empty_cache_path() 可以直接清空缓存
欢迎使用CZSC！当前版本标识为 0.10.3@20251003

任何个人、阶级，其力量归根结底来自自己，无须期盼或感恩于所谓的救世主、大救星。所有救世
主、大救星都不过是人造的，连神都是人造的，没有造的人，即使是神，也什么都不是！    

--摘自《2006-04-23 18:32 鼓吹救世主、大救星的是真正的精神鸦片！》
CZSC环境变量：czsc_min_bi_len = 6; czsc_max_bi_num = 50; 
C:\Users\zengb\.czsc 目录缓存超过1GB，请适当清理。调用 czsc.empty_cache_path() 可以直接清空缓存


2025-10-03 21:26:30.508 | INFO     | czsc.traders.cwc:init_tables:261 - metas 表创建成功！
2025-10-03 21:26:30.749 | INFO     | czsc.traders.cwc:init_tables:263 - weights 表创建成功！
2025-10-03 21:26:31.265 | INFO     | czsc.traders.cwc:init_tables:265 - returns 表创建成功！
2025-10-03 21:26:31.363 | INFO     | czsc.traders.cwc:init_latest_weights_view:161 - cs_latest_weights 视图初始化完成
2025-10-03 21:26:31.439 | INFO     | czsc.traders.cwc:init_latest_weights_view:188 - ts_latest_weights 视图初始化完成
2025-10-03 21:26:31.494 | INFO     | czsc.traders.cwc:init_latest_weights_view:198 - latest_weights 视图初始化完成
2025-10-03 21:26:31.495 | INFO     | czsc.traders.cwc:init_latest_weights_view:200 - 所有最新持仓权重视图初始化完成


### 策略数据写入

---

1. 写入策略元数据
2. 写入策略持仓权重
3. 写入策略日收益


In [2]:
czsc.cwc.set_meta(
    strategy="测试策略1号",
    base_freq="日线",
    description="这是一个测试用的策略",
    author="xxx",
    outsample_sdt="20220101",
    weight_type="ts",
    status="实盘",
    memo="",
    overwrite=False,
    database=database,
    tz=tz
)

mate = czsc.cwc.get_meta("测试策略1号", database=database)
print(mate)

2025-10-03 21:26:35.525 | WARNING  | czsc.traders.cwc:set_meta:367 - 策略 测试策略1号 已存在元数据，如需更新请设置 overwrite=True


{'strategy': '测试策略1号', 'base_freq': '日线', 'description': '这是一个测试用的策略', 'author': 'xxx', 'outsample_sdt': Timestamp('2022-01-01 00:00:00+0800', tz='Asia/Shanghai'), 'create_time': Timestamp('2025-10-03 20:24:39+0800', tz='Asia/Shanghai'), 'update_time': Timestamp('2025-10-03 20:24:39+0800', tz='Asia/Shanghai'), 'heartbeat_time': Timestamp('2025-10-03 21:00:03+0800', tz='Asia/Shanghai'), 'weight_type': 'ts', 'status': '实盘', 'memo': ''}


In [3]:
dfw = czsc.mock.generate_klines_with_weights()

czsc.cwc.publish_weights(df=dfw, strategy="测试策略1号", database=database, tz=tz)

2025-10-03 21:26:38.294 | INFO     | czsc.utils.cache:is_found:74 - 缓存文件已找到, C:\Users\zengb\.czsc\generate_klines\387FD5BE_generate_klines.pkl
2025-10-03 21:26:38.294 | INFO     | czsc.utils.cache:get:85 - 正在读取缓存记录，地址：C:\Users\zengb\.czsc\generate_klines\387FD5BE_generate_klines.pkl
2025-10-03 21:26:40.029 | INFO     | czsc.traders.cwc:__send_heartbeat:414 - 策略 测试策略1号 发送心跳成功
2025-10-03 21:26:40.580 | INFO     | czsc.traders.cwc:publish_weights:512 - 策略 测试策略1号 最新时间：2025-06-08 00:00:00+08:00
2025-10-03 21:26:40.594 | INFO     | czsc.traders.cwc:publish_weights:523 - 策略 测试策略1号 共 0 条新信号
2025-10-03 21:26:40.596 | INFO     | czsc.traders.cwc:publish_weights:531 - 准备发布 0 条策略信号
2025-10-03 21:26:40.597 | INFO     | czsc.traders.cwc:publish_weights:545 - 完成所有信号发布, 共 0 条
2025-10-03 21:26:40.780 | INFO     | czsc.traders.cwc:__send_heartbeat:414 - 策略 测试策略1号 发送心跳成功


In [4]:
wb = czsc.WeightBacktest(dfw, digits=2, fee_rate=0.0)
dfr = pd.melt(wb.daily_return.copy(), id_vars=["date"], var_name="symbol", value_name="returns")
dfr.rename(columns={"date": "dt"}, inplace=True)

print(dfr.tail())

czsc.cwc.publish_returns(df=dfr, strategy="测试策略1号", database=database, tz=tz)

                dt symbol   returns
101479  2025-06-04  total  0.005481
101480  2025-06-05  total  0.002237
101481  2025-06-06  total  0.003199
101482  2025-06-07  total -0.000736
101483  2025-06-08  total  0.000000


2025-10-03 21:26:49.450 | INFO     | czsc.traders.cwc:publish_returns:581 - 策略 测试策略1号 最新时间：2025-06-08 00:00:00+08:00
2025-10-03 21:26:49.467 | INFO     | czsc.traders.cwc:publish_returns:592 - 策略 测试策略1号 共 18 条新日收益
2025-10-03 21:26:49.470 | INFO     | czsc.traders.cwc:publish_returns:600 - 准备发布 18 条策略日收益
2025-10-03 21:26:49.682 | INFO     | czsc.traders.cwc:publish_returns:608 - 完成批次 1, 发布 18 条日收益
2025-10-03 21:26:49.683 | INFO     | czsc.traders.cwc:publish_returns:613 - 完成所有日收益发布, 共 18 条


### 读取策略数据
---

In [5]:
# 读取策略列表

dfm = czsc.cwc.get_all_metas(database=database)
dfm

,strategy,base_freq,description,author,outsample_sdt,create_time,update_time,heartbeat_time,weight_type,status,memo
0,测试策略1号,日线,这是一个测试用的策略,xxx,2022-01-01 00:00:00+08:00,2025-10-03 20:24:39+08:00,2025-10-03 20:24:39+08:00,2025-10-03 21:26:40+08:00,ts,实盘,


In [8]:
dfw = czsc.cwc.get_strategy_weights(strategy="测试策略1号", database=database)
dfw

,dt,symbol,weight,strategy,update_time
0,2010-01-01 00:00:00+08:00,000001,0.267271,测试策略1号,2025-10-03 20:24:44+08:00
1,2010-01-01 00:00:00+08:00,AAPL,-0.469111,测试策略1号,2025-10-03 20:24:44+08:00
2,2010-01-01 00:00:00+08:00,AMZN,-1.000000,测试策略1号,2025-10-03 20:24:44+08:00
3,2010-01-01 00:00:00+08:00,BTC,-0.921322,测试策略1号,2025-10-03 20:24:44+08:00
4,2010-01-01 00:00:00+08:00,CSCO,-1.000000,测试策略1号,2025-10-03 20:24:44+08:00
...,...,...,...,...,...
95841,2025-06-08 00:00:00+08:00,NVDA,0.565681,测试策略1号,2025-10-03 20:24:44+08:00
95842,2025-06-08 00:00:00+08:00,ORCL,-0.752165,测试策略1号,2025-10-03 20:24:44+08:00
95843,2025-06-08 00:00:00+08:00,PYPL,-1.000000,测试策略1号,2025-10-03 20:24:44+08:00
95844,2025-06-08 00:00:00+08:00,SAP,-0.853120,测试策略1号,2025-10-03 20:24:44+08:00


In [7]:
dfw = czsc.cwc.get_strategy_weights(strategy="测试策略1号", database=database, sdt="20230101")
dfw

,dt,symbol,weight,strategy,update_time
0,2023-01-01 00:00:00+08:00,000001,-0.539429,测试策略1号,2025-10-03 20:24:44+08:00
1,2023-01-01 00:00:00+08:00,AAPL,-0.477670,测试策略1号,2025-10-03 20:24:44+08:00
2,2023-01-01 00:00:00+08:00,AMZN,-1.000000,测试策略1号,2025-10-03 20:24:44+08:00
3,2023-01-01 00:00:00+08:00,BTC,-1.000000,测试策略1号,2025-10-03 20:24:44+08:00
4,2023-01-01 00:00:00+08:00,CSCO,-1.000000,测试策略1号,2025-10-03 20:24:44+08:00
...,...,...,...,...,...
15125,2025-06-08 00:00:00+08:00,NVDA,0.565681,测试策略1号,2025-10-03 20:24:44+08:00
15126,2025-06-08 00:00:00+08:00,ORCL,-0.752165,测试策略1号,2025-10-03 20:24:44+08:00
15127,2025-06-08 00:00:00+08:00,PYPL,-1.000000,测试策略1号,2025-10-03 20:24:44+08:00
15128,2025-06-08 00:00:00+08:00,SAP,-0.853120,测试策略1号,2025-10-03 20:24:44+08:00


In [9]:

dfr = czsc.cwc.get_strategy_returns(strategy="测试策略1号", database=database)
dfr

,dt,symbol,returns,strategy,update_time
0,2010-01-01 00:00:00+08:00,000001,0.0,测试策略1号,2025-10-03 20:25:05+08:00
1,2010-01-01 00:00:00+08:00,AAPL,0.0,测试策略1号,2025-10-03 20:25:05+08:00
2,2010-01-01 00:00:00+08:00,AMZN,0.0,测试策略1号,2025-10-03 20:25:05+08:00
3,2010-01-01 00:00:00+08:00,BTC,0.0,测试策略1号,2025-10-03 20:25:05+08:00
4,2010-01-01 00:00:00+08:00,CSCO,0.0,测试策略1号,2025-10-03 20:25:05+08:00
...,...,...,...,...,...
101479,2025-06-08 00:00:00+08:00,ORCL,0.0,测试策略1号,2025-10-03 21:26:49+08:00
101480,2025-06-08 00:00:00+08:00,PYPL,0.0,测试策略1号,2025-10-03 21:26:49+08:00
101481,2025-06-08 00:00:00+08:00,SAP,0.0,测试策略1号,2025-10-03 21:26:49+08:00
101482,2025-06-08 00:00:00+08:00,TSLA,0.0,测试策略1号,2025-10-03 21:26:49+08:00


In [10]:
dfl  = czsc.cwc.get_latest_weights(database=database)
dfl

,dt,symbol,weight,strategy,update_time
0,2025-06-08 00:00:00+08:00,000001,0.198232,测试策略1号,2025-10-03 20:24:44+08:00
1,2025-06-08 00:00:00+08:00,AAPL,-1.000000,测试策略1号,2025-10-03 20:24:44+08:00
2,2025-06-08 00:00:00+08:00,AMZN,-0.959428,测试策略1号,2025-10-03 20:24:44+08:00
3,2025-06-08 00:00:00+08:00,BTC,-1.000000,测试策略1号,2025-10-03 20:24:44+08:00
4,2025-06-08 00:00:00+08:00,CSCO,-1.000000,测试策略1号,2025-10-03 20:24:44+08:00
5,2025-06-08 00:00:00+08:00,ETH,-1.000000,测试策略1号,2025-10-03 20:24:44+08:00
6,2025-06-08 00:00:00+08:00,GOOGL,-1.000000,测试策略1号,2025-10-03 20:24:44+08:00
7,2025-06-08 00:00:00+08:00,IBM,-1.000000,测试策略1号,2025-10-03 20:24:44+08:00
8,2025-06-08 00:00:00+08:00,INTC,-0.664225,测试策略1号,2025-10-03 20:24:44+08:00
9,2025-06-08 00:00:00+08:00,META,-1.000000,测试策略1号,2025-10-03 20:24:44+08:00


In [11]:
tz = ZoneInfo("Asia/Tokyo")

dfl  = czsc.cwc.get_latest_weights(database=database, tz=tz)
print(dfl.tail())

dfr = czsc.cwc.get_strategy_returns(strategy="测试策略1号", database=database, tz=tz, sdt="20230101")
print(dfr.tail(), "\n", len(dfr), dfr.dt.min(), dfr.dt.max())

                          dt symbol    weight strategy               update_time
12 2025-06-08 01:00:00+09:00   NVDA  0.565681   测试策略1号 2025-10-03 21:24:44+09:00
13 2025-06-08 01:00:00+09:00   ORCL -0.752165   测试策略1号 2025-10-03 21:24:44+09:00
14 2025-06-08 01:00:00+09:00   PYPL -1.000000   测试策略1号 2025-10-03 21:24:44+09:00
15 2025-06-08 01:00:00+09:00    SAP -0.853120   测试策略1号 2025-10-03 21:24:44+09:00
16 2025-06-08 01:00:00+09:00   TSLA -1.000000   测试策略1号 2025-10-03 21:24:44+09:00
                             dt symbol  returns strategy               update_time
16015 2025-06-08 01:00:00+09:00   ORCL      0.0   测试策略1号 2025-10-03 22:26:49+09:00
16016 2025-06-08 01:00:00+09:00   PYPL      0.0   测试策略1号 2025-10-03 22:26:49+09:00
16017 2025-06-08 01:00:00+09:00    SAP      0.0   测试策略1号 2025-10-03 22:26:49+09:00
16018 2025-06-08 01:00:00+09:00   TSLA      0.0   测试策略1号 2025-10-03 22:26:49+09:00
16019 2025-06-08 01:00:00+09:00  total      0.0   测试策略1号 2025-10-03 22:26:49+09:00 
 16020 2023-01-

In [12]:
tz = ZoneInfo("America/Chicago")

dfl  = czsc.cwc.get_latest_weights(database=database, tz=tz)
print(dfl.tail())

dfr = czsc.cwc.get_strategy_returns(strategy="测试策略1号", database=database, tz=tz, sdt="20230101")
print(dfr.tail(), "\n", len(dfr), dfr.dt.min(), dfr.dt.max())

                          dt symbol    weight strategy               update_time
12 2025-06-07 11:00:00-05:00   NVDA  0.565681   测试策略1号 2025-10-03 07:24:44-05:00
13 2025-06-07 11:00:00-05:00   ORCL -0.752165   测试策略1号 2025-10-03 07:24:44-05:00
14 2025-06-07 11:00:00-05:00   PYPL -1.000000   测试策略1号 2025-10-03 07:24:44-05:00
15 2025-06-07 11:00:00-05:00    SAP -0.853120   测试策略1号 2025-10-03 07:24:44-05:00
16 2025-06-07 11:00:00-05:00   TSLA -1.000000   测试策略1号 2025-10-03 07:24:44-05:00
                             dt symbol  returns strategy               update_time
16015 2025-06-07 11:00:00-05:00   ORCL      0.0   测试策略1号 2025-10-03 08:26:49-05:00
16016 2025-06-07 11:00:00-05:00   PYPL      0.0   测试策略1号 2025-10-03 08:26:49-05:00
16017 2025-06-07 11:00:00-05:00    SAP      0.0   测试策略1号 2025-10-03 08:26:49-05:00
16018 2025-06-07 11:00:00-05:00   TSLA      0.0   测试策略1号 2025-10-03 08:26:49-05:00
16019 2025-06-07 11:00:00-05:00  total      0.0   测试策略1号 2025-10-03 08:26:49-05:00 
 16020 2022-12-